# Лабораторна робота No2
## Наука про дані: підготовчий етап

### Мета роботи: ознайомитися з основними кроками по роботі з даними – workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі.

In [1]:
import os
import pandas as pd
import urllib.request
from datetime import datetime
import numpy as np
import shutil
print("Завантажено.")

Завантажено.


#### 1. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження.

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def download():
    directory = 'VHI(lab2)'

    shutil.rmtree(directory)
    os.makedirs(directory, exist_ok=True)
    
    for i in range(1, 28):
        url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1982&year2=2023&type=Mean'.format(i)
        
        try:
            wp = urllib.request.urlopen(url)
            text = wp.read()
            
            now = datetime.now()
            date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
            
            filename = f'vhi_id={i}_{date_time}.csv'
            
            filepath = os.path.join(directory, filename)
            
            with open(filepath, 'wb') as out:
                out.write(text)
            
            print(f"VHI адмін. одиниці з індексом {i} збережено як {filename}.")
        except Exception as e:
            print(f"ПОМИЛКА ЗАВАНТАЖЕННЯ VHI адмін. одиниці з індексом {i}: {str(e)}")
download()

VHI адмін. одиниці з індексом 1 збережено як vhi_id=1_2024-03-22_10-48-55.csv.
VHI адмін. одиниці з індексом 2 збережено як vhi_id=2_2024-03-22_10-48-56.csv.
VHI адмін. одиниці з індексом 3 збережено як vhi_id=3_2024-03-22_10-48-57.csv.
VHI адмін. одиниці з індексом 4 збережено як vhi_id=4_2024-03-22_10-48-58.csv.
VHI адмін. одиниці з індексом 5 збережено як vhi_id=5_2024-03-22_10-48-59.csv.
VHI адмін. одиниці з індексом 6 збережено як vhi_id=6_2024-03-22_10-49-00.csv.
VHI адмін. одиниці з індексом 7 збережено як vhi_id=7_2024-03-22_10-49-01.csv.
VHI адмін. одиниці з індексом 8 збережено як vhi_id=8_2024-03-22_10-49-01.csv.
VHI адмін. одиниці з індексом 9 збережено як vhi_id=9_2024-03-22_10-49-02.csv.
VHI адмін. одиниці з індексом 10 збережено як vhi_id=10_2024-03-22_10-49-03.csv.
VHI адмін. одиниці з індексом 11 збережено як vhi_id=11_2024-03-22_10-49-04.csv.
VHI адмін. одиниці з індексом 12 збережено як vhi_id=12_2024-03-22_10-49-05.csv.
VHI адмін. одиниці з індексом 13 збережено як 

#### 2. Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [3]:
def read(directory):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    files = os.listdir(directory)
    result_df = pd.DataFrame()

    for i in range(len(files)):
        filepath = os.path.join(directory, files[i])
        df = pd.read_csv(filepath, header=1, names=headers, skiprows=1)  


        df = df.dropna(subset=['VHI'])
        df = df[df['VHI'] != -1]
          
        df = df.replace({'<tt>': '', '<pre>': ''})

        df['Area_ID'] = i + 1

        if 'empty' in df.columns:
            df.drop('empty', axis=1, inplace=True)

        if not df.empty:
            result_df = pd.concat([result_df, df], ignore_index=True)

    return result_df

directory = "VHI(lab2)"
df = read(directory)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  Area_ID
0      1982   2.0  0.034  251.07  40.82  71.85  56.33        1
1      1982   3.0  0.032  252.56  38.21  64.53  51.37        1
2      1982   4.0  0.030  254.76  32.74  55.45  44.10        1
3      1982   5.0  0.027  256.18  26.61  53.62  40.12        1
4      1982   6.0  0.025  256.78  20.92  56.90  38.91        1
...     ...   ...    ...     ...    ...    ...    ...      ...
57586  2023  48.0  0.077  263.87  40.72  50.93  45.82       27
57587  2023  49.0  0.067  262.62  37.92  52.13  45.02       27
57588  2023  50.0  0.062  261.76  38.51  49.47  43.99       27
57589  2023  51.0  0.058  261.06  40.45  45.06  42.76       27
57590  2023  52.0  0.056  260.25  41.33  43.52  42.43       27

[57591 rows x 8 columns]


#### 3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA.

In [4]:
replaced = {
    1: 'Вінницька', 2: 'Волинська', 3: 'Дніпропетровська', 4: 'Донецька',
    5: 'Житомирська', 6: 'Закарпатська', 7: 'Запорізька', 8: 'Івано-Франківська',
    9: 'Київська', 10: 'Кіровоградська', 11: 'Луганська', 12: 'Львівська',
    13: 'Миколаївська', 14: 'Одеська', 15: 'Полтавська', 16: 'Рівенська',
    17: 'Сумська', 18: 'Тернопільська', 19: 'Харківська', 20: 'Херсонська',
    21: 'Хмельницька', 22: 'Черкаська', 23: 'Чернівецька', 24: 'Чернігівська',
    25: 'Севастопіль' , 26: 'Київ', 27: 'Республіка Крим' }

df['Area_ID'] = df['Area_ID'].replace(replaced)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI          Area_ID
0      1982   2.0  0.034  251.07  40.82  71.85  56.33        Вінницька
1      1982   3.0  0.032  252.56  38.21  64.53  51.37        Вінницька
2      1982   4.0  0.030  254.76  32.74  55.45  44.10        Вінницька
3      1982   5.0  0.027  256.18  26.61  53.62  40.12        Вінницька
4      1982   6.0  0.025  256.78  20.92  56.90  38.91        Вінницька
...     ...   ...    ...     ...    ...    ...    ...              ...
57586  2023  48.0  0.077  263.87  40.72  50.93  45.82  Республіка Крим
57587  2023  49.0  0.067  262.62  37.92  52.13  45.02  Республіка Крим
57588  2023  50.0  0.062  261.76  38.51  49.47  43.99  Республіка Крим
57589  2023  51.0  0.058  261.06  40.45  45.06  42.76  Республіка Крим
57590  2023  52.0  0.056  260.25  41.33  43.52  42.43  Республіка Крим

[57591 rows x 8 columns]


#### 4. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
#### o Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [5]:
def extremes(df, area, year):
    min_vhi = df[(df['Area_ID'] == area) & (df['Year'] == year) & (df['VHI'] != -1)]['VHI'].min()
    max_vhi = df[(df['Area_ID'] == area) & (df['Year'] == year) & (df['VHI'] != -1)]['VHI'].max()
    return min_vhi, max_vhi
    
wanted_area = input("Адміністративна одиниця: ")
wanted_year = input("Рік: ")
wanted_year = str(wanted_year)

min_vhi, max_vhi = extremes(df, wanted_area, wanted_year)

def vhi_list(df, area, year):
    vhi_list = df.loc[(df['Area_ID'] == area) & (df['Year'] == year), ['Week', 'VHI']]
    return vhi_list
vhi_list = vhi_list(df, wanted_area, wanted_year)

print(f"Ряд VHI для адмін. одиниці {wanted_area} у {wanted_year} році:")
print(vhi_list)
print(f"Мінімальне значення VHI для адмін. одиниці {wanted_area} у {wanted_year} році: {min_vhi}")
print(f"Максимальне значення VHI для адмін. одиниці {wanted_area} у {wanted_year} році: {max_vhi}")

Адміністративна одиниця:  Київ
Рік:  2022


Ряд VHI для адмін. одиниці Київ у 2022 році:
       Week    VHI
55354   1.0  47.18
55355   2.0  46.85
55356   3.0  47.25
55357   4.0  50.95
55358   5.0  52.17
55359   6.0  50.15
55360   7.0  47.69
55361   8.0  46.05
55362   9.0  43.81
55363  10.0  44.73
55364  11.0  45.25
55365  12.0  44.57
55366  13.0  44.52
55367  14.0  44.39
55368  15.0  45.31
55369  16.0  46.48
55370  17.0  48.97
55371  18.0  51.48
55372  19.0  49.87
55373  20.0  49.02
55374  21.0  46.74
55375  22.0  42.65
55376  23.0  42.50
55377  24.0  41.36
55378  25.0  40.66
55379  26.0  40.96
55380  27.0  43.06
55381  28.0  47.24
55382  29.0  50.94
55383  30.0  54.63
55384  31.0  55.37
55385  32.0  55.41
55386  33.0  55.09
55387  34.0  53.32
55388  35.0  51.84
55389  36.0  51.15
55390  37.0  50.43
55391  38.0  49.56
55392  39.0  46.98
55393  40.0  47.02
55394  41.0  46.55
55395  42.0  40.29
55396  43.0  36.06
55397  44.0  32.05
55398  45.0  29.62
55399  46.0  31.95
55400  47.0  35.43
55401  48.0  40.12
55402  49.0  42.05
55403

#### o Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [6]:
def vhi_list(df, areas, start_year, end_year):
    result = pd.DataFrame()
    
    for area in areas:
        area_data = pd.DataFrame() 
        
        for year in range(start_year, end_year + 1):
            vhi_list = df.loc[(df['Area_ID'] == area) & (df['Year'] == str(year)), ['Week', 'VHI']]    
            vhi_list['Year'] = year
            vhi_list['Area'] = area      
            area_data = pd.concat([area_data, vhi_list], ignore_index=True)     
        result = pd.concat([result, area_data], ignore_index=True)    
    return result

selected_areas = input("Введіть адміністративні одиниці: ").split(',')
year1 = int(input("Введіть початковий рік: "))
year2 = int(input("Введіть кінцевий рік: "))

vhi_range_result = vhi_list(df, selected_areas, year1, year2)

print(f"Ряд VHI за діапазон років {year1} - {year2} для вказаних адмін. одиниць:")
print(vhi_range_result)


def drought_years(df, threshold_percentage):
    drought_years = []
    
    df_drought = df[(df['VHI'] <= 15) & (df['VHI'] != -1)]
    
    for year in df_drought['Year'].unique():
        df_year = df_drought[df_drought['Year'] == year]
        total_areas = df[df['Year'] == year]['Area_ID'].nunique()

        for area in df_year['Area_ID'].unique():
            df_area = df_year[df_year['Area_ID'] == area]
            drought_weeks = df_area['Week'].nunique()
            percentage = (drought_weeks / 52) * 100
            
            if percentage >= threshold_percentage:
                drought_years.append(year)
                break
    
    return drought_years

threshold_percentage = float(input("Введіть пороговий відсоток території,затронутої засухами: "))
drought_years_result = drought_years(df, threshold_percentage)

print(f"Роки, коли ексримальні засухи затронули більше {threshold_percentage}% областей:")
print(drought_years_result)

Введіть адміністративні одиниці:  Донецька,Севастопіль,Республіка Крим
Введіть початковий рік:  2006
Введіть кінцевий рік:  2012


Ряд VHI за діапазон років 2006 - 2012 для вказаних адмін. одиниць:
      Week    VHI  Year             Area
0      1.0  49.96  2006         Донецька
1      2.0  49.88  2006         Донецька
2      3.0  49.45  2006         Донецька
3      4.0  48.54  2006         Донецька
4      5.0  46.88  2006         Донецька
...    ...    ...   ...              ...
1087  48.0  47.63  2012  Республіка Крим
1088  49.0  50.77  2012  Республіка Крим
1089  50.0  53.09  2012  Республіка Крим
1090  51.0  51.17  2012  Республіка Крим
1091  52.0  48.78  2012  Республіка Крим

[1092 rows x 4 columns]


Введіть пороговий відсоток території,затронутої засухами:  1


Роки, коли ексримальні засухи затронули більше 1.0% областей:
['1986', '1993', '2000', '2007', '1984', '1994', '2012', '2003', '1999']


#### o Аналогічно для помірних посух

In [7]:
def light_drought_years(df, threshold_percentage):
    light_drought_years = []
    df_light_drought = df[(df['VHI'] > 15) & (df['VHI'] <= 35)]

    for year in df_light_drought['Year'].unique():
        df_year = df_light_drought[df_light_drought['Year'] == year]
        total_areas = df[df['Year'] == year]['Area_ID'].nunique()
        
        for area in df_year['Area_ID'].unique():
            df_area = df_year[df_year['Area_ID'] == area]
            light_drought_weeks = df_area['Week'].nunique()
            percentage = (light_drought_weeks / 52) * 100

            if percentage >= threshold_percentage:
                light_drought_years.append(year)
                break
    
    return light_drought_years

threshold_percentage = float(input("Введіть пороговий відсоток території, затронутої помірними посухами: "))
light_drought_years_result = light_drought_years(df, threshold_percentage)

print(f"Роки, в яких помірні посухи затронули більше {threshold_percentage}% областей:")
print(light_drought_years_result)


Введіть пороговий відсоток території, затронутої помірними посухами:  3


Роки, в яких помірні посухи затронули більше 3.0% областей:
['1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1999', '2000', '2002', '2003', '2006', '2007', '2008', '2010', '2011', '2012', '2015', '2017', '2018', '2019', '2020', '2021', '2023', '2001', '2009', '2016', '2013', '2004', '1998', '2005', '2014', '2022']
